In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
from binance import Client
from datetime import datetime 

import src.utils_binance as utils
import time


In [ ]:
client = utils.login_binance()
client.get_account_api_permissions()

In [ ]:
#client.get_account_snapshot(type='SPOT')

In [ ]:
def get_asset_balance(asset):
	asset_balance = client.get_asset_balance(asset)
	return float(asset_balance['free'])

def get_price_precision(minPrice: float):
	result = 8
	if minPrice >= 1.0:
		result = 0
	elif minPrice >= 0.1:
		result = 1
	elif minPrice >= 0.01:
		result = 2
	elif minPrice >= 0.001:
		result = 3
	elif minPrice >= 0.0001:
		result = 4
	elif minPrice >= 0.00001:
		result = 5
	elif minPrice >= 0.000001:
		result = 6
	elif minPrice >= 0.0000001:
		result = 7
	return result

In [ ]:
order_to_buy = 'XMRUSDT'
symbol_info = client.get_symbol_info(symbol=order_to_buy)
symbol_precision = int(symbol_info['baseAssetPrecision'])
print(f'symbol_info: {symbol_info}')
price_precision = get_price_precision(float(symbol_info['filters'][0]['tickSize']))
print(f'price_precision: {price_precision}')

In [ ]:
amount = 10.00
usdt_asset_balance = client.get_asset_balance('USDT')
print(usdt_asset_balance)
balance = float(usdt_asset_balance['free'])
if balance < 5.00:
	print(f'Not enough balance: {balance:.2f} USDT')
elif balance < amount and balance >= 5.00:
	amount = balance
print(f'Enough balance: {balance:.2f} USDT - Amount to buy: {amount:.2f} USDT')


In [ ]:
price_order = client.get_symbol_ticker(symbol=order_to_buy)
price = round(float(price_order['price']), symbol_precision)
print(price_order)
quantity = round(amount / price, 2)
print(f'quantity: {quantity}')

In [ ]:
order_buy_id = client.order_limit_buy(
	symbol=order_to_buy,
	quantity=quantity,
	price=str(price)
)

order_buy_id

In [ ]:
ticker = client.get_all_orders(symbol=order_to_buy)

for order in ticker:
	print(order, '\n')
	print(f"{order['symbol']} {order['price']} {order['origQty']} {order['side']} {order['type']} {order['status']}\n")

In [ ]:
margin = 0.015
multiplier = 2

price_precision = utils.get_precision(float(params['price_precision']))
take_profit = round(float(params['take_profit']), price_precision)
stop_loss_target = round(float(params['stop_loss']), price_precision)
stop_loss_trigger = round(stop_loss_target * 1.05, price_precision)
purchase_price = params['purchase_price']

filled_asset_balance = client.get_asset_balance(params['symbol'].split('USDT')[0])
int_quantity = filled_asset_balance['free'].split('.')[0]
frac_quantity = filled_asset_balance['free'].split('.')[1][:price_precision + 1]
quantity = float(int_quantity + '.' + frac_quantity)

oder_oco_sell_id = None
# oder_oco_sell_id = client.order_oco_sell(symbol=params['symbol'], quantity=quantity, price=str(take_profit), stopPrice=str(stop_loss_trigger), stopLimitPrice=str(stop_loss_target), stopLimitTimeInForce='GTC')

info_msg = f'ORDER SELL: symbol: {params["symbol"]} purchase_price: {purchase_price} price_precision: {price_precision} quantity: {quantity} take_profit: {take_profit} stop_loss_target: {stop_loss_target} stop_loss_trigger: {stop_loss_trigger}'
log.info(info_msg)
sm.send_to_telegram(info_msg)

log.info(f'oder_oco_sell_id: {oder_oco_sell_id}')